In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=3084131266768) was called 1 times


2020-12-24 11:57:50,842 - NumExpr defaulting to 8 threads.


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=3084131269360) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=3084131270512) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=3084131268352) was called 1 times
Running: Function "load_module_lookup" (id=3084131270080) was called 1 times
Running: Function "load_ID_lookup" (id=3084131268784) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [9]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av)

Running: Function "load_PTT_lookup" (id=3084131268496) was called 1 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3084131268208) was called 1 times
Running: Function "load_PTT_lookup" (id=3084131268496) was called 2 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3084131268208) was called 2 times
Running: Function "load_PTT_lookup" (id=3084131268496) was called 3 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3084131268208) was called 3 times
Running: Function "load_PTT_lookup" (id=3084131268496) was called 4 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3084131268208) was called 4 times
Running: Function "load_PTT_lookup" (id=3084131268496) was called 5 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=3084131268208) was called 5 times
Running: Function "load_PTT_lookup" (id=3084131268496) was called 6 times
Previous Hours Weighted
Running

In [10]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=1-y

In [14]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

#Problem with this function!

md.run_OLS(X_train,y_train,X_test,y_test, n = 30)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


339


TypeError: not all arguments converted during string formatting

### Add LM - metrics

In [ ]:
X_train_rd = X_train
Remove = 30

def find_features(X_train, y_train, X):
    max_p = 1
    while max_p > 0.1:
        model = sm.OLS(y_train, X_train)
        results = model.fit()
        top_X = results.pvalues.sort_values(ascending=False).head(X)
        max_p = top_X.tail(1).values[0]
        rm_col = list(results.pvalues.sort_values(ascending=False).head(X).index)
        X_train = X_train.drop(rm_col, axis=1)
    return X_train.columns

In [ ]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

In [ ]:
fa[fa['Fault ID']==7647]

In [ ]:
fa[fa['Pick Station']!=False]['Asset Code'].value_counts()

In [ ]:
pick_stations = ['PTT011','PTT012','PTT021','PTT022','PTT031','PTT032','PTT041','PTT042','PTT051','PTT052','PTT071','PTT072','PTT081','PTT082','PTT091','PTT092','PTT101','PTT102','PTT111','PTT112','PTT121','PTT122','PTT131','PTT132','PTT141','PTT142','PTT151','PTT152','PTT171','PTT172','PTT181','PTT182','PTT191','PTT192','PTT201','PTT202']

In [ ]:
fa_floor[fa_floor['Duration']==0]

In [ ]:
unmapped